# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part III: Training an End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a Bi-LSTM, a state-of-the-art deep neural network implemented in [TensorFlow](https://www.tensorflow.org/).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass:

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

We reload the probabilistic training labels:

In [3]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

We also reload the candidates:

In [4]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

Finally, we load gold labels for evaluation:

In [5]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now we can setup our discriminative model. Here we specify the model and learning hyperparameters.

They can also be set automatically using a search based on the dev set with a [GridSearch](https://github.com/HazyResearch/snorkel/blob/master/snorkel/learning/utils.py) object.

In [10]:
from snorkel.learning.disc_models.rnn import reRNN

train_kwargs = {
    'lr':         0.001,
    'dim':        100,
    'n_epochs':   20,
    'dropout':    0.5,
    'print_freq': 1,
    'max_sentence_length': 100
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[reRNN] Training model
[reRNN] n_train=17205  #epochs=20  batch size=256
[reRNN] Epoch 0 (46.90s)	Average loss=0.581320	Dev F1=6.39
[reRNN] Epoch 1 (97.85s)	Average loss=0.539535	Dev F1=33.73
[reRNN] Epoch 2 (149.13s)	Average loss=0.533978	Dev F1=37.90
[reRNN] Epoch 3 (200.61s)	Average loss=0.533322	Dev F1=38.72
[reRNN] Epoch 4 (252.64s)	Average loss=0.532603	Dev F1=36.73
[reRNN] Epoch 5 (301.74s)	Average loss=0.532447	Dev F1=37.10
[reRNN] Epoch 6 (354.16s)	Average loss=0.532263	Dev F1=38.33
[reRNN] Epoch 7 (403.45s)	Average loss=0.532098	Dev F1=38.54
[reRNN] Epoch 8 (454.91s)	Average loss=0.531792	Dev F1=39.47
[reRNN] Epoch 9 (504.96s)	Average loss=0.531588	Dev F1=39.49
[reRNN] Epoch 10 (559.52s)	Average loss=0.531594	Dev F1=39.45
[reRNN] Epoch 11 (635.33s)	Average loss=0.531617	Dev F1=39.26
[reRNN] Epoch 12 (716.63s)	Average loss=0.531529	Dev F1=39.48
[reRNN] Epoch 13 (787.75s)	Average loss=0.531207	Dev F1=40.42
[reRNN] Epoch 14 (843.61s)	Average loss=0.530980	Dev F1=41.89
[reRNN] Ep

Now, we get the precision, recall, and F1 score from the discriminative model:

In [11]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.363, Recall: 0.629, F1 Score: 0.461


We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [12]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.629
Neg. class accuracy: 0.9
Precision            0.363
Recall               0.629
F1                   0.461
----------------------------------------
TP: 141 | FP: 247 | TN: 2226 | FN: 83



Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

You can also improve performance substantially by increasing the number of training epochs!

Finally, we can save the predictions of the model on the test set back to the database. (This also works for other candidate sets, such as unlabeled candidates.)

In [13]:
lstm.save_marginals(session, test_cands)

Saved 2697 marginals


##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!